# Part 1
In the first part, we will learn how to work with unstructured data.
In particular, we will work with text data to a build language model (LM). 

LM is a probability distribution $p(\cdot)$ over some text $T$.
For example, if $T=(w_1\ w_2\ \dots \ w_{|T|})$, where $w_1,\ w_2\, \dots \ w_{|T|}$ is  collection of words and $|T|$ is the number of words in the text, LM is computed as
$p(T)=p(w_1\ w_2\ \dots \ w_{|T|})$

LM is employed in many applications such as automatic speech recognition (ASR), machine translation (MT), image captioning, text generation and etc. For example, LM is used in ASR to assess correctness of generated output transcripts.

Let's assume our ASR system generated two possible transcripts for some given speech signal: "recognize speech" and "wreck a nice beach".
These two phrases sound similar, but the first one is more likely to be linguistically correct.
We can use the probability of each transcript to determine the final output of ASR.
The probabilistic model (LM) should assign a higher probability score to the correct answer:

$p(``\text{recognize speech}") > p(``\text{wreck a nice beach}")$

### Reading a text file
We will use Penn Tree Bank ([PTB](https://catalog.ldc.upenn.edu/LDC99T42)) dataset as our text data in this assignment.

In [ ]:
# Reading a file into the string
with open('ptb.train.txt') as f:
   raw_data = f.read()

print("Type: " + str(type(raw_data)))  # type of the data
print("Size: " + str(len(raw_data)))   # number of characters
raw_data[967:1156]                     # print a small snippet of the data

Type: <class 'str'>
Size: 5101618


"although preliminary findings were reported more than a year ago the latest results appear in today 's new england journal of medicine a forum likely to bring new attention to the problem \n"

### Text preprocessing
Take a look at the PTB dataset. It is already pre-processed, i.e. words are tokenized, all letters are lowercased and etc. We need to apply a few small changes for our assignment. Please note, only the most frequient 10000 words are kept in the dataset, the rest of the words are replaced with <unk> symbol.This is done to reduce memory and computation requirements.   

In [ ]:
# Split the raw data into sentences and save them into a list (delimiter for sentence boundaries: '\n')
data = raw_data.split('\n')
data = [sent for sent in data if sent != ''] # remove empty sentences
print("Type: " + str(type(data)))
print("Size: " + str(len(data)))     # number of sentences
print("Sentence: " + data[2])        # print a sentence, try to print another sentence (symbol '<unk>' corresponds to unknown word)

Type: <class 'list'>
Size: 42068
Sentence:  mr. <unk> is chairman of <unk> n.v. the dutch publishing group 


In [ ]:
data[0]

' aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter '

### Your turn - more preprocessing

In [ ]:
# We need to add '<bos>' and '<eos>' symbols to the beginning and ending of each sentence respectively, e.g. "how are you"  => "<bos> how are you <eos>"
# To do so, first copy the data to data2 so that we do not modify data. Hint: use a function to perform a shallow copy
# Then make the changes to data2

stop = ('...', '.', '?', '!', '!!!')
data2 = data
data2 = ['<bos>'+sent+'<eos>' for sent in data2]



In [ ]:
# If you completed the task correctly, the first printed sentence will not have the inserted symbols
print("Sentence: " + data[0])
print("Sentence: " + data2[0])


Sentence:  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
Sentence: <bos> aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter <eos>


In [ ]:

words = [sent for sent in words if sent != '']
print(words)

IndentationError: unexpected indent (<ipython-input-6-cd0606148ffc>, line 2)

### Your turn - word counting

In [ ]:
# Create a dictionary which contains a word as a key and its frequency as a value, eg. 'judge': 262
# Hint: use split() function again to divide each sentence into a list of words
dictionary = dict()
#for i in range(len(data2)):
    #words = data2[i].split(' ')
    #words = [sent for sent in words if sent != '']
for i in range(len(data2)):
    words = data2[i].split()
    for word in words:
        if word in dictionary:
            dictionary[word]+=1
        else:
            dictionary[word]=1
print("The count of words in text: ", max(dictionary, key=dictionary.get))

The count of words in text:  the


In [ ]:
# If you completed the task correctly, the number of unique words shold be 10001 and the frequency of 'judge' should be 262
print("Size: "+str(len(dictionary))) # number of unique words
print(dictionary['judge']) # try other words

Size: 10001
262


### Building LM (probabilistic model)
To build a probabilistic model of a text, we can employ the frequentist approach, i.e. use relative frequency to estimate probability scores.
Suppose we want to estimate probability of a sentence "how are you", then we need to count how many times it appears in the dataset. This approach is infeasible for long and/or complex sentences that might be absent or appear a few times in the dataset.

Computing probability of short phrase is easy (click [here](https://books.google.com/ngrams/graph?content=How+are+you&year_start=1800&year_end=2000&corpus=15&smoothing=3&share=&direct_url=t1%3B%2CHow%20are%20you%3B%2Cc0#t1%3B%2CHow%20are%20you%3B%2Cc0)).
However, for a longer sentence it is difficult (click [here](https://books.google.com/ngrams/graph?content=I+met+my+friend+Madina&year_start=1800&year_end=2000&corpus=15&smoothing=3&share=&direct_url=))

#### Review: Chain rule
To circumvent the problem mentioned above, we will employ the chain rule to break down the joint probability of all words in a sentence into the sequence of conditional probabilities as follows:

$p(T)=p(w_1\ w_2\ \dots\ w_{|T|}) = p(w_1)\ p(w_2 | w_1)\ p(w_3 | w_1\ w_2)\ \dots \ p(w_{|T|} | w_1 \dots w_{|T|-1}) = p(w_1)\prod^{|T|}_{i=2}p(w_i|w_1\ \dots\ w_{i-1})$

where $p(w_3 | w_1\ w_2)$ - is the probability of word $w_3$ given that the preceeding two words in the sentence were $w_1$ and $w_2$.

For example, if $T= \text{"How are you"}$, then $w_1="How"$, $w_2 = "are"$ and $w_3 = "you"$, then the probability of having such sentence is $p(\text{"How are you"})= p("How")\ p("are"| "How")\ p("you" | "How" \ "are")$


The conditional probabilities are estimated as follows:

$p(w_i|w_{i-1})=\frac{count(w_{i-1}\ w_{i})}{count(w_{i-1})}$


Nevertheless, this is still infeasible for long sentences, because computing conditional probability of the last words still requires to count occurences of the preceding long phrases.

The simplest way to solve this problem is to treat each word independent from each other, i.e. $p(w_2|w_1)=p(w_2)$, $p(w_3| w_1\ w_2)=p(w_3)$ and so on (link to probability independence):

$p(T)=p(w_1\ w_2\ \dots\ w_{|T|}) \approx p(w_1)\ p(w_2)\ p(w_3)\ \dots \ p(w_{|T|}) = \prod^{|T|}_{i=1}p(w_i)$


The marginal probability of each word can be estimated as follows:

$p(w_i)=\frac{count(w_i)}{\sum count(w)} = \frac{count(w_i)}{Total\ number\ of\ words}$

### Your turn - estimating probability of a word
Create a dictionary called **dictionary_prob** which holds the marginal probability of each word.

In [ ]:
dictionary_prob = dict()
count = 0
for i in range(len(data2)):
    words = data2[i].split()
    count += len(words)
    for word in words:
        if word in dictionary:
            dictionary_prob[word] = dictionary[word]/sum(dictionary.values())            
print(dictionary_prob)

{'<bos>': 0.043295113399069836, 'aer': 1.0291697584641493e-06, 'banknote': 1.0291697584641493e-06, 'berlitz': 1.0291697584641493e-06, 'calloway': 1.0291697584641493e-06, 'centrust': 1.0291697584641493e-06, 'cluett': 1.0291697584641493e-06, 'fromstein': 1.0291697584641493e-06, 'gitano': 1.0291697584641493e-06, 'guterman': 1.0291697584641493e-06, 'hydro-quebec': 1.0291697584641493e-06, 'ipo': 1.0291697584641493e-06, 'kia': 1.0291697584641493e-06, 'memotec': 1.0291697584641493e-06, 'mlx': 1.0291697584641493e-06, 'nahb': 1.0291697584641493e-06, 'punts': 1.0291697584641493e-06, 'rake': 1.0291697584641493e-06, 'regatta': 1.0291697584641493e-06, 'rubens': 1.0291697584641493e-06, 'sim': 1.0291697584641493e-06, 'snack-food': 1.0291697584641493e-06, 'ssangyong': 1.0291697584641493e-06, 'swapo': 1.0291697584641493e-06, 'wachter': 1.0291697584641493e-06, '<eos>': 0.043295113399069836, 'pierre': 6.175018550784896e-06, '<unk>': 0.046333222526056005, 'N': 0.03342846292467404, 'years': 0.0012771996702

### Your turn - estimating probability of a sentence
Write a function called **compute_prob0** which takes a sentence and dictionary of marginal probabilites as input and returns its probability assuming each word is independent. For the words not present in the dictionary, probability of '\<unk>' symbol must be used. 


In [ ]:
def compute_prob0(sentence, dictionary_prob):
    prob = 1
    for word in sentence.split():
        if word in dictionary:
            prob = prob*dictionary_prob[word]
        else:
            prob = prob * dictionary_prob['<unk>']
    return prob

In [ ]:
# If you completed the task correctly, it should return 6.231238353466466e-11
sentence="hello how are you"
compute_prob0(sentence, dictionary_prob)


6.231238353466466e-11

In [ ]:
dictionary_prob['<unk>']

0.046333222526056005